# Long-Range Person Detection - Standard Training (YOLOv11)

This notebook runs the standard YOLOv11 training pipeline without SAHI.
It uses `pip` to install dependencies directly into the Colab environment.

In [1]:
# 1. Clone the Repository
!git clone https://github.com/haroon0x/y11-rfdetr.git
%cd y11-rfdetr

fatal: destination path 'y11-rfdetr' already exists and is not an empty directory.
/content/y11-rfdetr


In [2]:
# 2. Install Dependencies
# In Colab, we use pip directly. We don't need 'uv' virtual environments here.
!pip install -r requirements.txt
!pip install ultralytics

Ignoring colorama: markers 'platform_system == "Windows"' don't match your environment


In [14]:
# 3. Verify Environment
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
# Force delete the corrupted VisDrone dataset
!rm -rf /content/y11-rfdetr/yolo11m_training_runs/
print("Corrupted dataset deleted. You can now re-run the training cell.")

CUDA Available: True
Device: Tesla T4
Corrupted dataset deleted. You can now re-run the training cell.


In [11]:
# 4. Run Training Pipeline
# Defaulting to Nano (yolo11n.pt) as requested. Change to yolo11m.pt for Medium.
!python scripts/train_pipeline.py --model yolo11n.pt

Configuration: epochs=100, imgsz=1280, model=yolo11n.pt
=== Step 1: Training on VisDrone ===
Starting training on VisDrone.yaml with weights yolo11n.pt...
Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.1+cu128 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.3, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=VisDrone.yaml, degrees=10.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.5, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, n

In [13]:
# 5. Display Training Results (Graphs)
import os
from IPython.display import Image, display

# Find the latest training run directory
base_dir = "yolo11m_training_runs"
if os.path.exists(base_dir):
    steps = sorted(os.listdir(base_dir))
    if steps:
        latest_step = steps[-1]
        results_path = os.path.join(base_dir, latest_step, "results.png")
        confusion_matrix_path = os.path.join(base_dir, latest_step, "confusion_matrix.png")

        print(f"Displaying results for: {latest_step}")
        if os.path.exists(results_path):
            display(Image(filename=results_path))
        else:
            print("results.png not found.")

        if os.path.exists(confusion_matrix_path):
            display(Image(filename=confusion_matrix_path))
    else:
        print("No training steps found.")
else:
    print("Training output directory not found.")

Displaying results for: step1_visdrone
results.png not found.


In [6]:
# 6. Run Benchmark
# Benchmarks the best model from the first step (VisDrone) on the validation set
!yolo benchmark model=yolo11m_training_runs/step1_visdrone/weights/best.pt data=VisDrone.yaml imgsz=640 half=True device=0

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/__init__.py", line 955, in entrypoint
    model = YOLO(model, task=task)
            ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/models/yolo/model.py", line 76, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 144, in __init__
    self._load(model, task=task)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 283, in _load
    self.model, self.ckpt = load_checkpoint(weights)
                            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py", line 1461, in load_checkpoint
    ckpt, weight = torch_safe_load(weight)  # load ckpt
                   ^^^^^^^^^

In [7]:
# 7. Download Results
!zip -r runs_standard.zip yolo11m_training_runs/
from google.colab import files
files.download('runs_standard.zip')

updating: yolo11m_training_runs/ (stored 0%)
updating: yolo11m_training_runs/step1_visdrone/ (stored 0%)
updating: yolo11m_training_runs/step1_visdrone/args.yaml (deflated 52%)
updating: yolo11m_training_runs/step1_visdrone/weights/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>